## K-Core Sampling

- process files line by line and not try to read the whole thing into pandas. 
- Write four scripts, two each for users and items. 

1. Script A read the file line by line and output a table of users and their frequencies.
- Using the output of Script A, Script B reads the file again line by line and filters out the users with less than k items. 
2. Scripts C and D are similar to A and B, but for items. 
- Then you can run them A, B, C, D until the size of the file doesn't change. This should be relatively efficient.

In [11]:
# quickly make a copy of original data so I don't mess it up on accident!
# don't delete this cell
with open('final_track2_train.txt','r') as firstfile, open('k_core_ratings_0.txt','w') as secondfile:
    secondfile.truncate(0)
    # read content from first file
    for line in firstfile:
        # append content to second file
        secondfile.write(line)

In [27]:
#SCRIPT FOR CREATING K-CORE!!!
import os
k = 180
i = 0

while i < 30:
    if i > 1:
        print("i is: ", i)
        previous_file_size = os.path.getsize("k_core_ratings_{0}.txt".format(i-2))
        print("Previous file size was :", previous_file_size, "bytes")
        current_file_size = os.path.getsize("k_core_ratings_{0}.txt".format(i))
        print("Current file size is :", current_file_size, "bytes")
        if previous_file_size - current_file_size < 100:
            print("files are close enough in size for convergence!")
            print("final file name: k_core_ratings_{0}.txt".format(i))
            break
    #Script A read the file line by line and output a table of users and their frequencies.
    user_freq = {}
    with open("k_core_ratings_{0}.txt".format(i)) as fp:
        for line in fp:
            features = line.split()
            user_id = features[0]
            if user_id in user_freq:
                user_freq[user_id] += 1
            else:
                user_freq[user_id] = 1

    # Using the output of Script A, Script B reads the file again line by line 
    # and filters out the users with less than k items.

    file1 = open("k_core_ratings_{0}.txt".format(i+1), 'w')
    filteredOut = 0
    with open("k_core_ratings_{0}.txt".format(i)) as fp:
        for line in fp:
            # Writing to file
            features = line.split()
            user_id = features[0]
            if user_freq[user_id] < k:
                filteredOut+=1
            else:
                file1.writelines(line)

    file1.close()

    #Scripts C and D are similar to A and B, but for items.
    item_freq = {}
    with open("k_core_ratings_{0}.txt".format(i+1)) as fp:
        for line in fp:
            features = line.split()
            item_id = features[2]
            if item_id in item_freq:
                item_freq[item_id] += 1
            else:
                item_freq[item_id] = 1

    file1 = open("k_core_ratings_{0}.txt".format(i+2), 'w')
    filteredOut = 0
    with open("k_core_ratings_{0}.txt".format(i+1)) as fp:
        for line in fp:
            # Writing to file
            features = line.split()
            item_id = features[2]
            if item_freq[item_id] < k:
                filteredOut+=1
            else:
                file1.writelines(line)

    file1.close()
    i+=2

i is:  2
Previous file size was : 1069889124 bytes
Current file size is : 252900284 bytes
i is:  4
Previous file size was : 252900284 bytes
Current file size is : 138311583 bytes
i is:  6
Previous file size was : 138311583 bytes
Current file size is : 109636934 bytes
i is:  8
Previous file size was : 109636934 bytes
Current file size is : 98776472 bytes
i is:  10
Previous file size was : 98776472 bytes
Current file size is : 94347196 bytes
i is:  12
Previous file size was : 94347196 bytes
Current file size is : 92332392 bytes
i is:  14
Previous file size was : 92332392 bytes
Current file size is : 91502498 bytes
i is:  16
Previous file size was : 91502498 bytes
Current file size is : 90904131 bytes
i is:  18
Previous file size was : 90904131 bytes
Current file size is : 90560224 bytes
i is:  20
Previous file size was : 90560224 bytes
Current file size is : 90263556 bytes
i is:  22
Previous file size was : 90263556 bytes
Current file size is : 89998970 bytes
i is:  24
Previous file size

In [29]:
k_core_file = 28

In [30]:
#checking the length of the final dataset, and setting it for the creation of the
#different signal features
import pandas as pd
train_df = pd.read_csv("k_core_ratings_{0}.txt".format(k_core_file), sep='\t', names=['uid', 'user_city', 'item_id', 'author_id', 'item_city', 'channel', 'finish', 'like', 'music_id', 'did', 'create_time', 'video_duration'])
print("Length of final sampled training data: ", len(train_df))
train_df.head()

Length of final sampled training data:  1705250


,uid,user_city,item_id,author_id,item_city,channel,finish,like,music_id,did,create_time,video_duration
0,12533,148,1209100,504,37,0,1,0,597,32620,53086424288,30
1,32309,214,1209118,214956,7,0,0,0,-1,12812,53085854063,8
2,18091,6,1209136,23906,41,0,0,0,1954,2031,53086300944,10
3,20213,167,1209090,65069,7,0,1,0,-1,43959,53086464402,10
4,16311,57,7596,2191,41,0,0,0,578,15153,53086354469,9


# Explanation of Datasets Produced in this Notebook
#### In this notebook, I take two datasets from the ICME 2019 short video recommendation competition and compile them into several new datasets to fit the purposes of this research. The final datasets are samples of the original dataset, with k-core sampling on a k value of 180. 

I created 2 control datasets, one that used "users liking a video" as the positive signal, and one that used "users finishing a video" as the positive signal. These were used for the 2 control experiments, and then used again for testing for the experimental studies.

#### Previously, the original final datasets were going to be as follows:
1. User-Item-Ratings-Signal-1.csv: this is a dataset with the user, item, rating pairs, where the 'rating' is cacluated based on the following criteria: 
- 4.0 If a user finished and liked a video
- 3.0 if a use liked a video but did not finish it
- 2.0 if a user finished a video but did not like it
- 1.0 if a user did not like nor finish a video

2. User-Item-Ratings-Signal-2.csv: this is a dataset with the user, item, rating pairs, where the 'rating' is cacluated based on the following criteria: 
- 3.0 if a user finished and liked a video
- 2.0 if a user finished a video
- 1.0 otherwise

3. User-Item-Ratings-Signal-3.csv: this is a dataset with the user, item, rating pairs, where the 'rating' is cacluated based on the following criteria: 
- 3.0 if a user finished and liked a video
- 2.0 if a user liked a video
- 1.0 if otherwise

4. User-Item-Ratings-Signal-4.csv: this is a dataset with the user, item, rating pairs, where the 'rating' is cacluated based on the following criteria: 
- 2.0 if a user liked a video
- 1.0 if otherwise

4. User-Item-Ratings-Signal-5.csv: this is a dataset with the user, item, rating pairs, where the 'rating' is cacluated based on the following criteria: 
- 2.0 if a user finished a video
- 1.0 if otherwise

#### Additionally, in this notebook I also create a user features dataset and an item features dataset, both of which are compatible to be used with the librec-auto library for content-based filtering recommendation algorithms.

<hr/>

## Control 1: Train on Likes, Test on Likes

In [31]:
ratings_control_1 = train_df[['uid','item_id','like']]
for index,row in ratings_control_1.iterrows():
    if row[2] == 0:
        row[2] = -1
ratings_control_1.head()
# ratings_control_1.columns = ['user_id','item_id','rating']
# ratings_control_1.to_csv("ratings_control_1.csv",header=False, index=False)

,uid,item_id,like
0,12533,1209100,-1
1,32309,1209118,-1
2,18091,1209136,-1
3,20213,1209090,-1
4,16311,7596,-1


In [32]:
ratings_control_1.columns = ['user_id','item_id','rating']
len(ratings_control_1)

1705250

In [33]:
ratings_control_1.to_csv("ratings_control_1_small.csv",header=False, index=False)

## Control 2: Train on Finishes, Test on Finishes

In [34]:
ratings_control_2 = train_df[['uid','item_id','finish']]
for index,row in ratings_control_2.iterrows():
    if row[2] == 0:
        row[2] = -1
ratings_control_2.head()

,uid,item_id,finish
0,12533,1209100,1
1,32309,1209118,-1
2,18091,1209136,-1
3,20213,1209090,1
4,16311,7596,-1


In [35]:
ratings_control_2.columns = ['user_id','item_id','rating']
len(ratings_control_2)

1705250

In [10]:
ratings_control_2['rating'].value_counts()

-1    3695472
 1    2886559
Name: rating, dtype: int64

In [36]:
ratings_control_2.to_csv("ratings_control_2_small.csv",header=False, index=False)

In [152]:
ratings_control_2.head()

,user_id,item_id,rating
0,57384,43192,0
1,3230,46822,1
2,12637,269351,1
3,10987,65804,1
4,17534,1209085,0


## Signal 1: Taking likes and finishes into consideration

In [131]:
ratings = []
rating = 0
with open("k_core_ratings_{0}.txt".format(k_core_file)) as fp:
    for line in fp:
        features = line.split()
        finish = int(features[6])
        like = int(features[7])
        if finish == 1 and like == 1:
            rating = 4
        elif like == 1:
            rating = 3
        elif finish == 1:
            rating = 2
        else:
            rating = 1
        ratings.append(rating)

In [132]:
train_df['signal_1'] = ratings
train_df.head()

,uid,user_city,item_id,author_id,item_city,channel,finish,like,music_id,did,create_time,video_duration,signal_1,signal_2,signal_3,signal_4,signal_5
0,57384,52,43192,142828,0,0,0,0,4513,34178,53085993699,39,1,1,1,1,1
1,3230,5,46822,231026,1,0,1,0,5330,24878,53086372896,16,2,1,1,1,1
2,12637,265,269351,6757,6,0,1,0,-1,27794,53086427987,4,2,1,1,1,1
3,10987,115,65804,43245,10,0,1,0,7,17481,53085562259,21,2,1,1,1,1
4,17534,123,1209085,456222,11,0,0,0,-1,34312,53086389136,10,1,1,1,1,1


In [153]:
ratings_exp_1 = train_df[['uid','item_id','signal_1']]
ratings_exp_1.columns = ['user_id','item_id','rating']
ratings_exp_1.to_csv("ratings_exp_1.csv",header=False, index=False)

## Signal 2: Prioritizing 'finishing a video' as a positive signal

In [133]:
ratings_2 = []
with open("k_core_ratings_{0}.txt".format(k_core_file)) as fp:
    for line in fp:
        features = line.split()
        finish = int(features[6])
        like = int(features[7])
        if finish == 1 and like == 1:
            rating = 3
        elif finish == 1:
            rating = 2
        else:
            rating = 1
        ratings_2.append(rating)

In [134]:
train_df['signal_2'] = ratings_2
train_df.head()

,uid,user_city,item_id,author_id,item_city,channel,finish,like,music_id,did,create_time,video_duration,signal_1,signal_2,signal_3,signal_4,signal_5
0,57384,52,43192,142828,0,0,0,0,4513,34178,53085993699,39,1,1,1,1,1
1,3230,5,46822,231026,1,0,1,0,5330,24878,53086372896,16,2,2,1,1,1
2,12637,265,269351,6757,6,0,1,0,-1,27794,53086427987,4,2,2,1,1,1
3,10987,115,65804,43245,10,0,1,0,7,17481,53085562259,21,2,2,1,1,1
4,17534,123,1209085,456222,11,0,0,0,-1,34312,53086389136,10,1,1,1,1,1


In [163]:
ratings_exp_2 = train_df[['uid','item_id','signal_2']]
ratings_exp_2.columns = ['user_id','item_id','rating']
ratings_exp_2.to_csv("ratings_exp_2.csv",header=False, index=False)

## Signal 3: Prioritizing 'liking a video' as a positive signal

In [135]:
ratings_3 = []
with open("k_core_ratings_{0}.txt".format(k_core_file)) as fp:
    for line in fp:
        features = line.split()
        finish = int(features[6])
        like = int(features[7])
        if finish == 1 and like == 1:
            rating = 3
        elif like == 1:
            rating = 2
        else:
            rating = 1
        ratings_3.append(rating)

In [136]:
train_df['signal_3'] = ratings_3
train_df.head()

,uid,user_city,item_id,author_id,item_city,channel,finish,like,music_id,did,create_time,video_duration,signal_1,signal_2,signal_3,signal_4,signal_5
0,57384,52,43192,142828,0,0,0,0,4513,34178,53085993699,39,1,1,1,1,1
1,3230,5,46822,231026,1,0,1,0,5330,24878,53086372896,16,2,2,1,1,1
2,12637,265,269351,6757,6,0,1,0,-1,27794,53086427987,4,2,2,1,1,1
3,10987,115,65804,43245,10,0,1,0,7,17481,53085562259,21,2,2,1,1,1
4,17534,123,1209085,456222,11,0,0,0,-1,34312,53086389136,10,1,1,1,1,1


## Signal 4: Only signal is 'liking a video'

In [137]:
ratings_4 = []
with open("k_core_ratings_{0}.txt".format(k_core_file)) as fp:
    for line in fp:
        features = line.split()
        like = int(features[7])
        if like == 1:
            rating = 2
        else:
            rating = 1
        ratings_4.append(rating)

In [138]:
train_df['signal_4'] = ratings_4
train_df.head()

,uid,user_city,item_id,author_id,item_city,channel,finish,like,music_id,did,create_time,video_duration,signal_1,signal_2,signal_3,signal_4,signal_5
0,57384,52,43192,142828,0,0,0,0,4513,34178,53085993699,39,1,1,1,1,1
1,3230,5,46822,231026,1,0,1,0,5330,24878,53086372896,16,2,2,1,1,1
2,12637,265,269351,6757,6,0,1,0,-1,27794,53086427987,4,2,2,1,1,1
3,10987,115,65804,43245,10,0,1,0,7,17481,53085562259,21,2,2,1,1,1
4,17534,123,1209085,456222,11,0,0,0,-1,34312,53086389136,10,1,1,1,1,1


## Signal 5: Only signal is 'finishing a video'

In [146]:
ratings_5 = []
with open("k_core_ratings_{0}.txt".format(k_core_file)) as fp:
    for line in fp:
        features = line.split()
        finish = int(features[6])
        if finish == 1:
            rating = 2
        else:
            rating = 1
        ratings_5.append(rating)

In [147]:
train_df['signal_5'] = ratings_5
train_df.head()

,uid,user_city,item_id,author_id,item_city,channel,finish,like,music_id,did,create_time,video_duration,signal_1,signal_2,signal_3,signal_4,signal_5
0,57384,52,43192,142828,0,0,0,0,4513,34178,53085993699,39,1,1,1,1,1
1,3230,5,46822,231026,1,0,1,0,5330,24878,53086372896,16,2,2,1,1,2
2,12637,265,269351,6757,6,0,1,0,-1,27794,53086427987,4,2,2,1,1,2
3,10987,115,65804,43245,10,0,1,0,7,17481,53085562259,21,2,2,1,1,2
4,17534,123,1209085,456222,11,0,0,0,-1,34312,53086389136,10,1,1,1,1,1


<hr/>

In [148]:
train_df['signal_5'].value_counts()

1    3695472
2    2886559
Name: signal_5, dtype: int64